In [1]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

In [5]:
#import useful functions from sripts folder
import sys
sys.path.append("../scripts/")
from features import feature_scaling
data = pd.read_csv("../data/features2.csv")
data.head()

,diagnosis,radius_worst,perimeter_worst,area_worst,concave_points_mean,concave_points_worst
0,M,25.38,184.60,2019.0,0.7119,0.2654
1,M,24.99,158.80,1956.0,0.2416,0.1860
2,M,23.57,152.50,1709.0,0.4504,0.2430
3,M,14.91,98.87,567.7,0.6869,0.2575
4,M,22.54,152.20,1575.0,0.4000,0.1625


In [7]:

scaled_features = feature_scaling(data,'diagnosis')
scaled_features.head()

,diagnosis,radius_worst,perimeter_worst,area_worst,concave_points_mean,concave_points_worst
0,1,1.886690,2.303601,2.001237,2.109526,2.296076
1,1,1.805927,1.535126,1.890489,-0.146749,1.087084
2,1,1.511870,1.347475,1.456285,0.854974,1.955000
3,1,-0.281464,-0.249939,-0.550021,1.989588,2.175786
4,1,1.298575,1.338539,1.220724,0.613179,0.729259


In [8]:

import statsmodels.api as sm
X = data.drop(['diagnosis'],axis=1)
y = data['diagnosis']

smlog = sm.Logit(y,X).fit()
smlog.summary()

Optimization terminated successfully.
         Current function value: 0.114796
         Iterations 10


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:              diagnosis   No. Observations:                  569
Model:                          Logit   Df Residuals:                      564
Method:                           MLE   Df Model:                            4
Date:                Wed, 25 Aug 2021   Pseudo R-squ.:                  0.8261
Time:                        18:34:48   Log-Likelihood:                -65.319
converged:                       True   LL-Null:                       -375.72
Covariance Type:            nonrobust   LLR p-value:                4.875e-133
========================================================================================
                           coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------
radius_worst            -1.4151      0.480     -2.947      0.003      -2.356      -0.474
perimeter_worst         -0.0837      0.072     -1.168      0.243      -0.224       0.057
area_worst               0.0302      0.004      8.039      0.000       0.023       0.038
concave_points_mean      1.1190      1.707      0.655      0.512      -2.227       4.466
concave_points_worst    51.8906     10.152      5.111      0.000      31.993      71.788
========================================================================================

Possibly complete quasi-separation: A fraction 0.20 of observations can be
perfectly predicted. This might indicate that there is complete
quasi-separation. In this case some parameters will not be identified.
"""


when the p-value is greater than 0.05, we fail to reject the null hypothesis that the relationship between the patient's diagnosis and the exogenous variable is not significant.

In [9]:
data.shape

(569, 6)

In [11]:
range(1,6)

range(1, 6)

In [12]:

data.columns.to_list()

['diagnosis',
 'radius_worst',
 'perimeter_worst',
 'area_worst',
 'concave_points_mean',
 'concave_points_worst']

In [14]:

#area and perimeter functions of radius
from causalgraphicalmodels import CausalGraphicalModel
causal = CausalGraphicalModel(
    nodes=['radius_worst','perimeter_worst','area_worst','concave_points_mean','diagnosis','concave_points_worst'],
    edges=[
        ("radius_worst", "perimeter_worst"), 
        ("radius_worst", "area_worst"),
        ("diagnosis", "area_worst"),
        ("concave_points_worst", "area_worst"),    
        ("concave_points_mean", "diagnosis")
    ]
)
# draw return a graphviz `dot` object, which jupyter can render
causal.draw()

ModuleNotFoundError: No module named 'causalgraphicalmodels'